# Introducing py2neo

py2neo is the most popular of the Python drivers used to interact with Neo4j. For simplicity, this example assumes that you've got authentication turned off. 

You can turn authentication off by uncommenting this line in your neo4j.conf file:

`dbms.security.auth_enabled=false`

Now we'll import py2neo and write a simple query to find all the groups that have 'Python' in the name:

In [ ]:
from py2neo import Graph
graph = Graph()

In [ ]:
query = """
MATCH (group:Group)-[:HAS_TOPIC]->(topic)
WHERE group.name CONTAINS "Python" 
RETURN group.name, COLLECT(topic.name) AS topics
"""

result = graph.cypher.execute(query)

for row in result:
    print(row) 

You should see a few groups and a list of the topics that they have.

# Calculating topic similarity

Now that we've got the hang of executing Neo4j queries from Python let's calculate topic similarity based on common groups so that we can use it in our queries.

We'll first import the igraph library:

In [ ]:
from igraph import Graph as IGraph

Next we'll write a query which finds all pairs of topics and then works out the number of common groups. We'll use that as our 'weight' in the similarity calculation.

In [ ]:
query = """
MATCH (topic:Topic)<-[:HAS_TOPIC]-()-[:HAS_TOPIC]->(other:Topic)
WHERE ID(topic) < ID(other)
RETURN topic.name, other.name, COUNT(*) AS weight
ORDER BY weight DESC
LIMIT 10
"""

graph.cypher.execute(query)

Now let's run the query again and wrap the output in igraph:

In [ ]:
query = """
MATCH (topic:Topic)<-[:HAS_TOPIC]-()-[:HAS_TOPIC]->(other:Topic)
WHERE ID(topic) < ID(other)
RETURN topic.name, other.name, COUNT(*) AS weight
"""

ig = IGraph.TupleList(graph.cypher.execute(query), weights=True)
ig

We're now ready to run a community detection algorithm over the graph to see what clusters/communities we have:

In [ ]:
clusters = IGraph.community_walktrap(ig, weights="weight")
clusters = clusters.as_clustering()
len(clusters)

Let's have a quick look at what we've got:

In [ ]:
nodes = [node["name"] for node in ig.vs]
nodes = [{"id": x, "label": x} for x in nodes]
nodes[:5]

for node in nodes:
    idx = ig.vs.find(name=node["id"]).index
    node["group"] = clusters.membership[idx]
    
nodes[:5]

And finally we're going to write a Cypher query which takes the results of our community detection algorithm and writes the results back into Neo4j:

In [ ]:
query = """
UNWIND {params} AS p 
MATCH (t:Topic {name: p.id}) 
MERGE (cluster:Cluster {name: p.group})
MERGE (t)-[:IN_CLUSTER]->(cluster)
"""

graph.cypher.execute(query, params = nodes)